<a href="https://colab.research.google.com/github/NicolasFerre/ColabFerre/blob/dev/mercado_libre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerías necesarias

In [176]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
from openpyxl.utils import get_column_letter, column_index_from_string

In [177]:
archivo = 'Resultados ML.xlsx'

# Archivos necesarios

In [178]:
dfm = pd.read_excel('MercadoLibre.xlsx', sheet_name=0, header=5)
renglonaje = pd.read_excel('Renglonaje.xlsx', sheet_name=0, header=0)
envios = pd.read_excel('Envios.xlsx', sheet_name=0, header=3)

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


## Extracción de precios del renglonaje

In [179]:
renglonaje.rename(columns={'Total que paga Edy a Truper contando reserva en garantía CON IVA': 'Costo a Truper'}, inplace=True)
precios_sku = renglonaje[['clave', 'Costo a Truper']]
precios_sku = precios_sku.loc[precios_sku['Costo a Truper'] != 'No disponible']
precios_sku['Costo a Truper'] = precios_sku['Costo a Truper'].astype(float)

In [180]:
precios_sku.rename(columns={'clave': 'SKU'}, inplace=True)

In [181]:
precios_codigo = renglonaje[['código', 'Costo a Truper']]
precios_codigo = precios_codigo.loc[precios_codigo['Costo a Truper'] != 'No disponible']
precios_codigo['Costo a Truper'] = precios_codigo['Costo a Truper'].astype(float)
precios_codigo.rename(columns={'código': 'SKU'}, inplace=True)
precios_codigo['SKU'] = precios_codigo['SKU'].astype(str)

## Extracción de costos de envío

In [182]:
orden = envios['NUMERO ORDEN']
orden = orden.str.split('-')

In [183]:
orden = orden.str[1]

In [184]:
envios['NUMERO ORDEN'] = orden

In [185]:
envios['NUMERO ORDEN'] = envios['NUMERO ORDEN'].replace('', np.nan)
envios = envios.dropna(subset=['NUMERO ORDEN'])

In [186]:
publicacion = envios[['NUMERO ORDEN', 'VALOR GUIA ($)', 'ZONA EXTENDIDA ($)', 'EXTRAGRANDE ($)', 'SOBREPESOS ($)', 'TOTAL GUIA ($)']]

In [187]:
publicacion.rename(columns={'NUMERO ORDEN': '# de venta'}, inplace=True)

/tmp/ipython-input-187-1123234388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  publicacion.rename(columns={'NUMERO ORDEN': '# de venta'}, inplace=True)


# Costo que Edgar paga a Truper

In [188]:
df = dfm.iloc[:,:15]
df2 = dfm.iloc[:,15:]
df2['# de venta'] = dfm['# de venta'].astype(str)

In [189]:
df[['Ingresos por envío (MXN)', 'Costos de envío (MXN)']] = df[['Ingresos por envío (MXN)', 'Costos de envío (MXN)']].fillna(0)

In [190]:
df = df.merge(precios_sku, on='SKU', how='left')
df = df.merge(precios_codigo, on='SKU', how='left')

In [191]:
df[['Costo a Truper_x', 'Costo a Truper_y']] = df[['Costo a Truper_x', 'Costo a Truper_y']].fillna(0)

In [192]:
df['Costo a Truper'] = df['Costo a Truper_x'] + df['Costo a Truper_y']
df = df.drop(['Costo a Truper_x', 'Costo a Truper_y'], axis=1)

In [193]:
df['Costo a Truper'] = df['Costo a Truper'] * df['Unidades']

# Costo de la guía Skydropx

In [194]:
df['# de venta'] = df['# de venta'].astype(str)

In [195]:
df = df.merge(publicacion, on='# de venta', how='left')

# Calculo de rentabilidad

In [196]:
df.iloc[:,15:] = df.iloc[:,15:].fillna(0)

In [197]:
df['TOTAL GUIA ($)']

,TOTAL GUIA ($)
0,0
1,0
2,0
3,0
4,0
...,...
1760,0
1761,0
1762,0
1763,0


In [198]:
df['TOTAL GUIA ($)'] = df['TOTAL GUIA ($)'].replace('-', 0)
df['TOTAL GUIA ($)'] = df['TOTAL GUIA ($)'].astype(float)

/tmp/ipython-input-198-2228787780.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['TOTAL GUIA ($)'] = df['TOTAL GUIA ($)'].replace('-', 0)


In [199]:
df['Rentabilidad'] = df['Total (MXN)'] - df['Costo a Truper'] - df['TOTAL GUIA ($)']

In [200]:
df.loc[df['Costo a Truper'] == 0, 'Rentabilidad'] = 0

In [201]:
df.loc[df['Anulaciones y reembolsos (MXN)'] < 0, 'Rentabilidad'] = 0

In [202]:
df['¿Rentable?'] = ''
df.loc[df['Rentabilidad'] > 0, '¿Rentable?'] = 'Sí'
df.loc[df['Rentabilidad'] <= 0, '¿Rentable?'] = 'No'

In [203]:
df = df.merge(df2, on='# de venta', how='left')

# Limpieza de datos

In [204]:
df = df.drop_duplicates(subset=['# de venta'])

# Creacion del excel

In [205]:
df.to_excel(archivo, sheet_name='Rentabilidad', index=False)

# Formato con Openpyxl

In [206]:
wb = load_workbook(archivo)
ws = wb['Rentabilidad']

## Columnas por índice

In [207]:
total = column_index_from_string('M')
costo = column_index_from_string('P')
envio = column_index_from_string('U')
rentabilidad = column_index_from_string('V')
rentable = column_index_from_string('W')
c_final = column_index_from_string('BO')

## Máximo de filas

In [208]:
max_rows = 3500

## Colores de fondo

En orden, todos con tonalidades claras:
- naranja
- verde
- rojo
- azul
- morado
- gris
- amarillo
- negro

In [209]:
fill_o = PatternFill(start_color='f9cb9c', end_color='f9cb9c', fill_type='solid')
fill_lg = PatternFill(start_color='b6d7a8', end_color='b6d7a8', fill_type='solid')
fill_lr = PatternFill(start_color='ea9999', end_color='ea9999', fill_type='solid')
fill_lb = PatternFill(start_color='a4c2f4', end_color='a4c2f4', fill_type='solid')
fill_p = PatternFill(start_color='b4a7d6', end_color='b4a7d6', fill_type='solid')
fill_gy = PatternFill(start_color='cccccc', end_color='cccccc', fill_type='solid')
fill_y = PatternFill(start_color='ffe599', end_color='ffe599', fill_type='solid')
fill_b = PatternFill(start_color='000000', end_color='000000', fill_type='solid')

## Fuentes

In [210]:
font_w = Font(name='Calibri', size=12, color = 'ffffff', bold=True)
font_b = Font(name='Calibri', size=12, color = '000000', bold=True)
font_t = Font(name='Calibri', size=12, color = '000000', bold=False)

## Dando formato

### Color a columnas

In [211]:
for row in range(1, max_rows):
  if ws.cell(row=row, column=rentable).value == None:
    break
  ws.cell(row=row, column=total).fill = fill_y
  ws.cell(row=row, column=costo).fill = fill_o
  ws.cell(row=row, column=envio).fill = fill_lb
  for col in range(costo + 1, envio):
    ws.cell(row=row, column=col).fill = fill_p

### Color rentabilidad

In [212]:
for row in range(2, max_rows):
  if ws.cell(row=row, column=rentabilidad).value == None:
    break
  if ws.cell(row=row, column=rentabilidad).value > 0:
    ws.cell(row=row, column=rentabilidad).fill = fill_lg
    ws.cell(row=row, column=rentable).fill = fill_lg
  else:
    ws.cell(row=row, column=rentabilidad).fill = fill_lr
    ws.cell(row=row, column=rentable).fill = fill_lr

### Fuente a columnas

### Formato numerico

In [213]:
num_format = '$ #,##0.00'

In [214]:
for row in range(2, max_rows):
  for col in range(8, 12):
    ws.cell(row=row, column=col).number_format = num_format
  ws.cell(row=row, column=total).number_format = num_format
  for col in range(costo, rentabilidad+1):
    ws.cell(row=row, column=col).number_format = num_format

### Claridad visual del documento

Eliminamos los valores 0 de las columnas de Total, Costo a Truper, valores de envío y rentabilidad.

In [215]:
for row in range(2, max_rows):
  for col in range(8, 22):
    if ws.cell(row=row, column=col).value == 0:
      ws.cell(row=row, column=col).value = None

Le ponemos la fuente a las cabezas de las columnas

In [216]:
for col in range(1, c_final):
  ws.column_dimensions[get_column_letter(col)].width = 14
  ws.cell(row=1, column=col).font = font_b

Coloreamos la cabeza de las columnas de Rentabilidad y Rentable

In [217]:
ws.cell(row=1, column=rentabilidad).fill = fill_b
ws.cell(row=1, column=rentabilidad).font = font_w
ws.cell(row=1, column=rentable).fill = fill_b
ws.cell(row=1, column=rentable).font = font_w

Le ponemos fuente a todo el documento

In [218]:
for row in range(2, max_rows):
  for col in range(1, c_final):
    ws.cell(row=row, column=col).font = font_t

# Guardando archivo

In [219]:
wb.save(archivo)